In [ ]:
import yfinance as yf
import datetime as dt
import requests, json, pandas as pd
from bs4 import BeautifulSoup
from pyspark.sql import *
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType
import pandas as pd
import os
from delta import configure_spark_with_delta_pip, DeltaTable

# Get environment variables from Railway
SPARK_MASTER_URL = os.getenv('SPARK_MASTER_URL')
SPARK_MASTER_HOST = os.getenv('SPARK_MASTER_HOST')

# Initialize the Spark session builder with Railway cluster configuration
builder = SparkSession.builder \
    .appName("MyApp1") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.master", SPARK_MASTER_URL) \
    .config("spark.driver.host", SPARK_MASTER_HOST) \
    .config("spark.driver.port", "7337")

# Add required packages including the new Delta packages
packages = [
    "io.delta:delta-spark_2.12:3.1.0",
    "io.delta:delta-storage:3.1.0",
    "io.delta:delta-contribs_2.12-3.1.0"
]
builder.config("spark.jars.packages", ",".join(packages))

# Create the Spark session
spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
import os
import psycopg2

def connect_using_separate_vars():
    try:
        # Get individual environment variables
        host = os.getenv('PGHOST')
        database = os.getenv('PGDATABASE')
        user = os.getenv('PGUSER')
        password = os.getenv('PGPASSWORD')
        port = os.getenv('PGPORT')
        
        if not all([host, database, user, password, port]):
            raise Exception("Missing required database environment variables")
            
        # Connect to the database
        conn = psycopg2.connect(
            host=host,
            database=database,
            user=user,
            password=password,
            port=port
        )
        
        return conn

    except Exception as e:
        print(f"Error connecting to database: {e}")
        return None

def test_connection():
    # Try connecting using separate variables
    conn = connect_using_separate_vars()
    
    if conn:
        try:
            cursor = conn.cursor()
            
            # Test the connection
            cursor.execute('SELECT version();')
            version = cursor.fetchone()
            print(f"Successfully connected to PostgreSQL. Version: {version[0]}")
            
            # Example query
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS test_table (
                    id SERIAL PRIMARY KEY,
                    name VARCHAR(100)
                );
            """)
            conn.commit()
            print("Test table created successfully!")
            
        except Exception as e:
            print(f"Error executing query: {e}")
            
        finally:
            cursor.close()
            conn.close()
            print("Database connection closed.")
    else:
        print("Failed to connect to database")

if __name__ == "__main__":
    test_connection()